<a href="https://colab.research.google.com/github/jyryu3161/SyntheticBiology/blob/main/Lec4_ligandMPNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting up Condacolab in Google Colab

In [1]:
!pip install -q condacolab # install the condacolab package
import condacolab # Import and initialize condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check() # verification of the installation

✨🍰✨ Everything looks OK!


Now you can use Conda commands directly in your notebook:

In [2]:
!git clone https://github.com/dauparas/LigandMPNN.git
!conda create -n ligandmpnn_env python=3.9 -y


Cloning into 'LigandMPNN'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 338 (delta 32), reused 22 (delta 22), pack-reused 278 (from 2)
Receiving objects: 100% (338/338), 2.62 MiB | 19.30 MiB/s, done.
Resolving deltas: 100% (133/133), done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/ligandmpnn_env

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    ld_impl_linux-64-2.43      |       h71

In [1]:
!pip3 install six==1.16.0


In [2]:
import os
os.chdir('/content/LigandMPNN/')

In [3]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 90.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.4 MB/s eta 0:00:00
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 154.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 159.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━

In [4]:
!bash get_model_params.sh "./model_params"


In [ ]:
# {
# "C1": "ACDEFGHIKLMNPQRSTVW",
# "C3": "ACDEFGHIKLMNPQRSTVW",
# "C5": "ACDEFGHIKLMNPQRSTVW",
# "C7": "ACDEFGHIKLMNPQRSTVW"
# }
!python run.py \
        --seed 111 \
        --pdb_path "./inputs/1BC8.pdb" \
        --omit_AA_per_residue "./inputs/omit_AA_per_residue.json" \
        --redesigned_residues "C1 C2 C3 C4 C5 C6 C7 C8 C9 C10" \
        --out_folder "./test_outputs/per_residue_omit" \
        --batch_size 3 \
        --number_of_batches 5

### Enzyme Engineering Examples

- pdb_id = "1LDM"
- ligand_resname = "OXM"

https://www.rcsb.org/structure/1LDM

In [12]:
from Bio.PDB import *
import numpy as np

def find_nearby_residues(pdb_file, ligand_name, distance_cutoff=5.0):
    # PDB 파서 초기화
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    # 지정된 리간드 찾기
    ligand_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                # 지정된 리간드 식별 (HET 레코드)
                if residue.resname == ligand_name:
                    ligand_atoms = list(residue.get_atoms())
                    break

    if not ligand_atoms:
        print(f"{ligand_name} 리간드를 찾을 수 없습니다.")
        return []

    # 가까운 잔기 찾기
    nearby_residues = set()
    for ligand_atom in ligand_atoms:
        for model in structure:
            for chain in model:
                for residue in chain:
                    # HET 레코드나 수용 원자 제외
                    if residue.resname == ligand_name or residue.id[0] != ' ':
                        continue

                    # 잔기의 모든 원자에 대해 거리 계산
                    for atom in residue:
                        distance = ligand_atom - atom
                        if distance <= distance_cutoff:
                            nearby_residues.add(residue)
                            break

    # 결과 출력 및 반환
    print(f"{ligand_name} 리간드 주변 {distance_cutoff}Å 이내 잔기:")
    for residue in nearby_residues:
        print(f"- {residue.resname} {residue.id[1]} (Chain {residue.parent.id})")
    print('--------')
    for residue in nearby_residues:
        print("%s%s"%(residue.parent.id, residue.id[1]), end=" ")
    return list(nearby_residues)

# 사용 예시
pdb_file = './inputs/1ldm.pdb'  # 실제 PDB 파일 경로로 대체
nearby_residues = find_nearby_residues(pdb_file, 'OXM')

OXM 리간드 주변 5.0Å 이내 잔기:
- HIS 193 (Chain A)
- ALA 235 (Chain A)
- ARG 169 (Chain A)
- ARG 106 (Chain A)
- ASN 138 (Chain A)
- LEU 165 (Chain A)
- THR 245 (Chain A)
- ILE 239 (Chain A)
- GLN 100 (Chain A)
--------
A193 A235 A169 A106 A138 A165 A245 A239 A100 

In [13]:
!python run.py \
        --seed 111 \
        --pdb_path "./inputs/1ldm.pdb" \
        --omit_AA_per_residue "./inputs/omit_AA_per_residue.json" \
        --redesigned_residues "A193 A235 A169 A106 A138 A165 A245 A239 A100" \
        --out_folder "./test_outputs/enz_eng_example" \
        --batch_size 3 \
        --number_of_batches 10

Designing protein from this path: ./inputs/1ldm.pdb
These residues will be redesigned:  ['A100', 'A106', 'A138', 'A165', 'A169', 'A193', 'A235', 'A239', 'A245']
These residues will be fixed:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67', 'A68', 'A69', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79', 'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A90', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99', 'A101', 'A102', 'A103', 'A104', 'A105', 'A107', 'A108', 'A109', 'A110', 'A111', 'A112', 'A113', 'A114', 'A115', 'A116', 'A1